In [26]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from lxml import html
import requests as req
from locale import *

url_base = "https://uspdigital.usp.br/tycho/curriculoLattesBuscar?codpes="
xpath = "//*[@id=\"layout_conteudo\"]/table[1]"

In [27]:
raw = pd.read_csv("votos.tsv", delimiter="\t")

In [31]:
raw.NUSP.value_counts()

8761313    2
7580306    2
5440511    2
1234567    2
9796381    1
4649690    1
8049711    1
93***90    1
8945005    1
7581905    1
7994711    1
666666     1
7581680    1
8059553    1
9301325    1
7161837    1
8627017    1
9796568    1
8125021    1
9790728    1
7182195    1
7160774    1
8944901    1
6434781    1
9797444    1
6362284    1
7579051    1
9999999    1
7580286    1
9796811    1
          ..
9366381    1
8539537    1
4911424    1
7161048    1
9848944    1
8538950    1
9318459    1
5212562    1
5644202    1
1370277    1
9796954    1
8577801    1
9302368    1
7511844    1
8539728    1
9796593    1
8068463    1
8944411    1
5914412    1
8540122    1
6800000    1
9074375    1
9796512    1
8889497    1
7777312    1
9302006    1
8642842    1
9302121    1
9796450    1
9301951    1
Name: NUSP, dtype: int64

In [9]:
lista_valido = []
for index, voto in raw.iterrows():
    if 1 == 1:
        url = url_base + voto.NUSP
        page = req.get(url)
        tree = html.fromstring(page.content)
        resul = tree.xpath(xpath)
        try:
            resul_el = resul[0][1][1].text
        except IndexError:
            lista_valido.append("nao")
            continue
            
        lista_valido.append("sim")
print("Acabou")

Acabou


In [10]:
lista_IF = []
for index, voto in raw.iterrows():
    if voto.Valido == "sim":
        url = url_base + voto.NUSP
        page = req.get(url)
        tree = html.fromstring(page.content)
        resul = tree.xpath(xpath)
        try:
            resul_el = resul[0][1][1].text
        except IndexError:
            lista_IF.append("?")
            continue
            
        condicao = (("Instituto de Física" in resul_el) or
                    ("Interunidades FE/IF/IQ/IB" in resul_el) or
                    ("Interunidades Pró-G" in resul_el))
        
        if condicao:
            lista_IF.append("sim")
        else:
            lista_IF.append("não")
    else:
        lista_IF.append("N/A")
print("Acabou")

Acabou


In [23]:
df1 = pd.DataFrame(lista_IF, columns=["IF"])
df2 = pd.DataFrame(lista_valido, columns=["ValidoTycho"])
resultado = pd.concat([raw, df1], axis=1)
resultado = pd.concat([resultado, df2], axis=1)

In [24]:
resultado.to_csv("resultado.csv")

In [25]:
resultado[~(resultado.Valido == resultado.ValidoTycho)]

,Data/hora,Greve?,Piquete?,NUSP,NUSP corrigo,last,Valido,Motivo,IF,ValidoTycho
45,6/16/2016 22:40:59,Não,Não,8945113,89***13,Gui,sim,NaN,?,nao
100,6/17/2016 0:44:47,Não,Não,7182195,71***95,Ito,nao,digitos,N/A,sim
137,6/17/2016 10:51:07,Sim,Não,7512042,75***42,131,nao,digitos,N/A,sim
172,6/17/2016 22:42:35,Sim,Sim,7994663,79***63,ira,sim,NaN,?,nao
